In [1]:
import numpy as np
import pandas as pd

In [2]:
import io 
%cd "C:\Users\ykkou\Desktop\imarticus\Tutorial Datasets"

C:\Users\ykkou\Desktop\imarticus\Tutorial Datasets


In [3]:
df = pd.read_csv('master-catalogue-mertrics - Amazon-products-meta.csv')

In [4]:
df = df.transpose()

In [5]:
df.reset_index(inplace=True)

In [6]:
df.columns = df.iloc[0]

In [7]:
df = df[1:]

In [8]:
df

,Product name,Product description,Brand name,Quantity,All products images/ videos,Rating,Number of Ratings,MRP,Selling price,Product URL,...,Skin Type,Brand,Scent,Material Feature,Material Type Free,Age Range (Description),Number of Items,Net Quantity,Special Feature,Highly rated by customers for
1,Unnamed: 1,NaN,NaN,grams / ml,Best quality,NaN,NaN,NaN,NaN,NaN,...,if available,if available,if available,if available,if available,if available,if available,if available,if available,all data
2,Unnamed: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unnamed: 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import requests
from bs4 import BeautifulSoup


In [10]:
custom_headers = {
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
'accept language' : 'en-US,en;q=0.9'
}

In [11]:
url = 'https://www.amazon.in/s?rh=n%3A1374407031&fs=true&ref=lp_1374407031_sar'



In [16]:
import time

max_retries = 5
retry_delay = 7  

for attempt in range(max_retries):
    response = requests.get(url, headers = custom_headers)
    if response.status_code == 200:
       
        break
    elif response.status_code == 503:
        
        print(f"Received a 503 status code. Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
    else:
        
        print(f"Received status code {response.status_code}. Exiting...")
        break


Received a 503 status code. Retrying in 7 seconds...


In [17]:
print(response.status_code)

200


In [18]:
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
product_links = soup.find_all('a', class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")

In [20]:
relative_links = []
prod_desc = []

In [21]:
span_elements = soup.find_all('span', class_='a-size-base-plus a-color-base a-text-normal')

# Extract and print the text content of each span element
for span_element in span_elements:
    text_content = span_element.text.strip()
    prod_desc.append(text_content)

In [22]:
for link in product_links:
    relative_links.append(link.get('href'))


In [23]:
from urllib.parse import urljoin

In [24]:
base_url = "https://amazon.in"

In [25]:
complete_links = [urljoin(base_url, relative_link) for relative_link in relative_links] 

In [26]:
images = []

In [27]:
img_elements = soup.find_all('img', class_='s-image')

# Extract and print the source (URL) of each img element
for img_element in img_elements:
    image_source = img_element['src']
    images.append(image_source)

In [28]:
no_ratings = []

In [29]:
span_elements = soup.find_all('span', class_='a-size-base s-underline-text')

# Extract and print the text content of each span element
for span_element in span_elements:
    text_content = span_element.text.strip()
    no_ratings.append(text_content)


In [33]:
span_elements = soup.find_all('span', class_='a-price-whole')

# Extract the numbers and store them in the selling_price list
selling_price = []

for span_element in span_elements:
    text_content = span_element.text.strip()
    selling_price.append(text_content)


In [44]:
print(len(complete_links),
len(prod_desc),
len(images),
len(no_ratings),len(selling_price))


48 48 48 48 48


In [43]:
# Quality Check to be performed on the images scrap. But Assuming correct order for now and pushing into the dataframe
images = images[:48]

In [41]:
num_empty_rows = 45

# Create empty rows with NaN values
empty_rows = pd.DataFrame(np.nan, index=range(num_empty_rows), columns=df.columns)

# Append the empty rows to the existing DataFrame
df = pd.concat([df, empty_rows], ignore_index=True)


In [45]:
df['Product URL'] = complete_links

In [46]:
df['Product description'] = prod_desc

In [47]:
df['All products images/ videos'] = images

In [48]:
df['Number of Ratings'] = no_ratings

In [49]:
df['Selling price '] = selling_price

In [50]:
df

,Product name,Product description,Brand name,Quantity,All products images/ videos,Rating,Number of Ratings,MRP,Selling price,Product URL,...,Skin Type,Brand,Scent,Material Feature,Material Type Free,Age Range (Description),Number of Items,Net Quantity,Special Feature,Highly rated by customers for
0,Unnamed: 1,Dettol Original Germ Protection Bathing Soap B...,NaN,grams / ml,https://m.media-amazon.com/images/I/61a5LNKUAZ...,NaN,"21,288",NaN,225,https://amazon.in/Dettol-Original-Protection-B...,...,if available,if available,if available,if available,if available,if available,if available,if available,if available,all data
1,Unnamed: 2,Dettol Original Germ Protection Bathing Soap B...,NaN,NaN,https://m.media-amazon.com/images/I/616Jgnn6KE...,NaN,68,NaN,124,https://amazon.in/Dettol-Original-Soap-75g-Pac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unnamed: 3,Cetaphil Face Wash Gentle Skin Cleanser for Dr...,NaN,NaN,https://m.media-amazon.com/images/I/71t9JRry+3...,NaN,"25,845",NaN,276,https://amazon.in/Cetaphil-Gentle-Skin-Cleanse...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Ice Face Roller Silicone Facial Cube for Eyes ...,NaN,NaN,https://m.media-amazon.com/images/I/11gNbuzzir...,NaN,"1,737",NaN,49,https://amazon.in/Roller-Silicone-Facial-Massa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Dettol Liquid Handwash Refill – Skincare Hand ...,NaN,NaN,https://m.media-amazon.com/images/I/61DKLhAwWu...,NaN,"73,014",NaN,193.,https://amazon.in/Dettol-pH-Balanced-Skincare-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Minimalist 2% Salicylic Acid Face Wash For Oil...,NaN,NaN,https://m.media-amazon.com/images/I/61J2gr1xbL...,NaN,"10,178",NaN,299,https://amazon.in/Minimalist-Salicylic-Sulphat...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,"Himalaya Purifying Neem Face Wash, 400 Ml",NaN,NaN,https://m.media-amazon.com/images/I/51qpGr-1qz...,NaN,"88,101",NaN,268,https://amazon.in/Himalaya-Purifying-Neem-Face...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Minimalist Sunscreen SPF 50 Lightweight with M...,NaN,NaN,https://m.media-amazon.com/images/I/51tXjE2WQQ...,NaN,"14,221",NaN,399,https://amazon.in/Minimalist-Sunscreen-Multi-V...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Park Avenue Good Morning Soap Pack of 4 500gm,NaN,NaN,https://m.media-amazon.com/images/I/611+X0ZGdW...,NaN,"27,656",NaN,195,https://amazon.in/Park-Avenue-Good-Morning-500...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,The Derma Co 1% Hyaluronic Sunscreen Aqua Ultr...,NaN,NaN,https://m.media-amazon.com/images/I/81TIzxoGl4...,NaN,"10,551",NaN,438,https://amazon.in/Derma-Co-Hyaluronic-Sunscree...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.to_csv('master-catalogue-mertrics - Amazon-products-meta.csv',index=False)